In [14]:
import numpy as np
import pandas as pd

Loads title.akas.tsv data

In [15]:
title_akas = pd.read_csv('/Users/seankamano/Downloads/title.akas.tsv', delimiter = '\t', encoding = 'utf-8')
print title_akas.head(10)

     titleId  ordering                      title region language  \
0  tt0000001         1  Carmencita - spanyol tánc     HU       \N   
1  tt0000001         2                 Карменсита     RU       \N   
2  tt0000001         3                 Carmencita     US       \N   
3  tt0000001         4                 Carmencita     \N       \N   
4  tt0000002         1     Le clown et ses chiens     \N       \N   
5  tt0000002         2          A bohóc és kutyái     HU       \N   
6  tt0000002         3     Le clown et ses chiens     FR       \N   
7  tt0000002         4     The Clown and His Dogs     US       \N   
8  tt0000002         5         Клоун и его собаки     RU       \N   
9  tt0000003         1            Szegény Pierrot     HU       \N   

         types             attributes isOriginalTitle  
0  imdbDisplay                     \N               0  
1           \N                     \N               0  
2           \N                     \N               0  
3     original  

Need to load title.basics.tsv data to get movies only. Gets tconst for all movies.

In [16]:
title_basics = pd.read_csv('/Users/seankamano/Downloads/title.basics.tsv', delimiter='\t', encoding='utf-8')
title_basics_movie = title_basics[title_basics['titleType'] == 'movie']
uid = title_basics_movie['tconst']

print uid.head(10)

8      tt0000009
145    tt0000147
332    tt0000335
499    tt0000502
571    tt0000574
611    tt0000615
626    tt0000630
669    tt0000675
670    tt0000676
673    tt0000679
Name: tconst, dtype: object


Get movies only from title.akas.tsv by taking uid and comparing them to the tconsts in title.akas.tsv. This filters out any entries that are not movies in title.akas.tsv

In [17]:
title_akas_movies = title_akas[title_akas['titleId'].isin(uid.values)]

print title_akas_movies.head(10)

        titleId  ordering                          title region language  \
46    tt0000009         1                     Miss Jerry     \N       \N   
47    tt0000009         2                     Miss Jerry     HU       \N   
48    tt0000009         3                     Miss Jerry     US       \N   
568   tt0000147         1  The Corbett-Fitzsimmons Fight     US       \N   
971   tt0000335         1        Early Christian Martyrs     AU       \N   
972   tt0000335         2          Soldiers of the Cross     AU       \N   
973   tt0000335         3          Soldiers of the Cross     \N       \N   
1447  tt0000502         1                       Bohemios     ES       \N   
1663  tt0000574         1     Kelly bandájának története     HU       \N   
1664  tt0000574         2    The Story of the Kelly Gang     \N       \N   

            types attributes isOriginalTitle  
46       original         \N               1  
47    imdbDisplay         \N               0  
48             \N     

Remove movies without any regional data. Store movies without regional data for later reference, if it's ever needed.

In [18]:
title_akas_movies_by_region = title_akas_movies[title_akas_movies['region'] != '\N']
title_akas_movies_no_region = title_akas_movies[title_akas_movies['region'] == '\N']
print title_akas_movies_by_region.head(10)
print '\n',"*"*100,'\n'
print title_akas_movies_no_region.head(10)

        titleId  ordering                           title region language  \
47    tt0000009         2                      Miss Jerry     HU       \N   
48    tt0000009         3                      Miss Jerry     US       \N   
568   tt0000147         1   The Corbett-Fitzsimmons Fight     US       \N   
971   tt0000335         1         Early Christian Martyrs     AU       \N   
972   tt0000335         2           Soldiers of the Cross     AU       \N   
1447  tt0000502         1                        Bohemios     ES       \N   
1663  tt0000574         1      Kelly bandájának története     HU       \N   
1665  tt0000574         3  Die Geschichte der Kelly Bande     DE       \N   
1666  tt0000574         4     The Story of the Kelly Gang     AU       \N   
1783  tt0000615         1              Robbery Under Arms     AU       \N   

            types attributes isOriginalTitle  
47    imdbDisplay         \N               0  
48             \N         \N               0  
568        

Need to make sense of region codes. Scrape data off of https://help.imdb.com/article/contribution/other-submission-guides/country-codes/G99K4LFRMSC37DCN#

In [136]:
import urllib2
from bs4 import BeautifulSoup

soup = BeautifulSoup(urllib2.urlopen('https://help.imdb.com/article/contribution/other-submission-guides/country-codes/G99K4LFRMSC37DCN#').read(), "lxml")

tableClassResults = soup.find("table",{"summary":"List of accepted country codes - by country"})
countries = tableClassResults.find_all('li')
other_countries = soup.find_all('tr')
#other_countries contains codes for countries which no longer exist (USSR, Yugoslavia, etc.)

list_countries = [country.contents[0][4:] for country in countries]
list_codes = [str(country.contents[0][0:2]).upper() for country in countries]
other_countries = other_countries[-5:]
for country in other_countries:
    list_codes.append(str(country.contents[0])[4:8].upper())
for country in other_countries:
    list_countries.append(str(country.contents[1]))
print list_countries[:10]
print list_codes[:10]

[u'Afghanistan', u'\xc5land Islands', u'Albania', u'Algeria', u'American Samoa', u'Andorra', u'Angola', u'Anguilla', u'Antarctica', u'Antigua and Barbuda']
['AF', 'AX', 'AL', 'DZ', 'AS', 'AD', 'AO', 'AI', 'AQ', 'AG']


Zip codes with country names so we have a reference for what each two-character acronym represents.

In [139]:
country_dict = dict(zip(list_codes,list_countries))

for key in country_dict:
    if key[0] == 'A':
        print key,country_dict[key]


AF Afghanistan
AW Aruba
AE United Arab Emirates
AD Andorra
AG Antigua and Barbuda
AI Anguilla
AM Armenia
AL Albania
AO Angola
AN Netherlands Antilles
AQ Antarctica
AS American Samoa
AR Argentina
AU Australia
AT Austria
AX Åland Islands
AZ Azerbaijan


We now a more recognizable form for each country code. Now to find how many movies have been released in each country. 

In [143]:
movies_per_country = dict(zip(list_codes, [0 for i in range(len(list_codes))]))
for code in title_akas_movies_by_region['region']:
    try:
        movies_per_country[str(code)] += 1
    except(KeyError):
        continue

for key in movies_per_country:
    if key[0] == 'A':
        print key,movies_per_country[key]


AF 77
AW 7
AE 129
AD 11
AG 11
AI 1
AM 184
AL 954
AO 36
AN 7
AQ 4
AS 2
AR 17519
AU 8740
AT 11654
AX 0
AZ 424


Now for the average rating of all movies seen by a given country. Need title.ratings.tsv, extract movies with regional codes from it, and then attach a rating to each title.

In [144]:
title_ratings = pd.read_csv('/Users/seankamano/Downloads/title.ratings.tsv', delimiter='\t', encoding='utf-8')
title_ratings.head(10)

,tconst,averageRating,numVotes
0,tt0000001,5.8,1350
1,tt0000002,6.5,157
2,tt0000003,6.6,933
3,tt0000004,6.4,93
4,tt0000005,6.2,1621
5,tt0000006,5.7,80
6,tt0000007,5.5,543
7,tt0000008,5.6,1441
8,tt0000009,5.4,62
9,tt0000010,6.9,4845


Get the tconsts in title.ratings.tsv

In [145]:
tconsts = title_ratings['tconst'].values
ratings = title_ratings['averageRating'].values
tconst_rating_pairs = dict(zip(tconsts,ratings))
tconst_rating_pairs

{u'tt7763324': 7.2000000000000002,
 u'tt1810373': 7.0999999999999996,
 u'tt1810372': 7.5,
 u'tt1810371': 7.2000000000000002,
 u'tt0411539': 6.2000000000000002,
 u'tt0411538': 7.9000000000000004,
 u'tt2877896': 8.5,
 u'tt0382269': 4.7999999999999998,
 u'tt3393762': 7.5,
 u'tt0382267': 7.2000000000000002,
 u'tt2836260': 4.7000000000000002,
 u'tt0382262': 7.0,
 u'tt0382263': 5.5999999999999996,
 u'tt0382261': 5.9000000000000004,
 u'tt1525340': 7.2000000000000002,
 u'tt1525343': 5.7999999999999998,
 u'tt4706200': 5.9000000000000004,
 u'tt1525344': 7.2000000000000002,
 u'tt0299018': 7.5,
 u'tt0299019': 6.0999999999999996,
 u'tt0299016': 8.0,
 u'tt0299017': 6.7000000000000002,
 u'tt0299014': 5.5,
 u'tt0299015': 6.5,
 u'tt0372962': 6.5,
 u'tt0299013': 6.2000000000000002,
 u'tt0299010': 4.7999999999999998,
 u'tt2975946': 6.2999999999999998,
 u'tt0525191': 7.5,
 u'tt0525193': 6.9000000000000004,
 u'tt1979319': 7.5999999999999996,
 u'tt1074026': 8.6999999999999993,
 u'tt1074021': 6.2999999999999

Get all movies with a valid region and cross reference with movies which have an average rating.

In [149]:
movies_region_and_rating = title_akas_movies_by_region[title_akas_movies_by_region['titleId'].isin(tconsts)]
movies_region_and_rating.head(10)

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
47,tt0000009,2,Miss Jerry,HU,\N,imdbDisplay,\N,0
48,tt0000009,3,Miss Jerry,US,\N,\N,\N,0
568,tt0000147,1,The Corbett-Fitzsimmons Fight,US,\N,\N,\N,0
971,tt0000335,1,Early Christian Martyrs,AU,\N,working,\N,0
972,tt0000335,2,Soldiers of the Cross,AU,\N,\N,\N,0
1663,tt0000574,1,Kelly bandájának története,HU,\N,imdbDisplay,\N,0
1665,tt0000574,3,Die Geschichte der Kelly Bande,DE,\N,\N,\N,0
1666,tt0000574,4,The Story of the Kelly Gang,AU,\N,\N,\N,0
1783,tt0000615,1,Robbery Under Arms,AU,\N,\N,\N,0
1820,tt0000630,1,Amleto,IT,\N,\N,\N,0


Need to get ratings and connect them to country codes. First get ratings from title.ratings.tsv and attach them with their respective tconsts. Then, by comparing tconsts with titleId in movies_region_and_rating, get the country code.

In [ ]:
titleIds = movies_region_and_rating['titleId'].values
regions = movies_region_and_rating['region'].values
titleId_region_pairs = dict(zip(titleIds,regions))
titleId_region_pairs

In [166]:
ratings_per_country_stats = dict(zip(list_codes, [[0,0] for i in range(len(list_codes))]))
ratings_per_country = dict(zip(list_codes, [0 for i in range(len(list_codes))]))
for tcode in movies_region_and_rating['titleId']:
    try:
        tmp_rating = tconst_rating_pairs[tcode] 
        tmp_ccode = titleId_region_pairs[tcode]
        ratings_per_country_stats[tmp_ccode][0] += tmp_rating
        ratings_per_country_stats[tmp_ccode][1] += 1
    except KeyError:
        continue

for key in ratings_per_country:
    try:
        ratings_per_country[key] = ratings_per_country_stats[key][0] / ratings_per_country_stats[key][1]
    except ZeroDivisionError:
        ratings_per_country[key] = 'N/A'

for key in ratings_per_country:
    if key[0] == 'A':
        print key,ratings_per_country[key]

print '*'*100
for key in ratings_per_country_stats:
    if key[0] == 'A':
        print key,ratings_per_country_stats[key]

AF 7.3
AW 7.2
AE 6.90243902439
AD N/A
AG 7.66666666667
AI N/A
AM 5.97109375
AL 6.56267409471
AO 6.73333333333
AN 8.8
AQ N/A
AS N/A
AR 6.46875083233
AU 6.35048785154
AT 6.57792853444
AX N/A
AZ 7.3725848564
****************************************************************************************************
AF [204.40000000000003, 28]
AW [14.4, 2]
AE [282.99999999999994, 41]
AD [0, 0]
AG [23.0, 3]
AI [0, 0]
AM [764.30000000000018, 128]
AL [2355.9999999999964, 359]
AO [40.399999999999999, 6]
AN [8.8000000000000007, 1]
AQ [0, 0]
AS [0, 0]
AR [97147.700000000987, 15018]
AU [33193.999999999927, 5227]
AT [63509.900000000103, 9655]
AX [0, 0]
AZ [2823.7000000000035, 383]
